<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/model/modelTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model.py


In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary

In [0]:
from Model import Model

In [6]:
model=Model()
print(model)
summary(model,input_size=(3,32,32))

Model(
  (model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=False)
    (5): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Dropout(p=0.5, inplace=False)
    (9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (12): ReLU()
    (13): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (14): ReLU()
    (15): AdaptiveAvgPool2d(output_size=1)
    (16): Flatten()
    (17): Softmax(dim=None)
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1), stride=(1,

RuntimeError: ignored

In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [10]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

--2020-03-06 13:17:01--  https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4519 (4.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.41K  --.-KB/s    in 0s      

2020-03-06 13:17:01 (53.7 MB/s) - ‘utils.py’ saved [4519/4519]



In [11]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
Files already downloaded and verified


In [21]:
for i in range(3):
  for j in range(4):
    baseModel=[False]*3
    modifiedModel=[False]*4
    baseModel[i]=True
    modifiedModel[j]=True
    # Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
    # Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
    # The scecond one is the strided. The third: ConvPool. The last: All CNN.
    try:
      del model
    except:
      print('')

    model = Model(baseModel=baseModel,modifiedModel=modifiedModel)  
    
    #if cuda:
    #    model.cuda()
    model.cpu()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[200, 250, 300], gamma=0.1)

    #
    summary(model.model, input_size=(3, 32, 32),device='cpu')
    print(model)
    

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Dropout-1            [-1, 3, 32, 32]               0
            Conv2d-2           [-1, 96, 30, 30]           7,296
              ReLU-3           [-1, 96, 30, 30]               0
         MaxPool2d-4           [-1, 96, 14, 14]               0
           Dropout-5           [-1, 96, 14, 14]               0
            Conv2d-6          [-1, 192, 12, 12]         460,992
              ReLU-7          [-1, 192, 12, 12]               0
         MaxPool2d-8            [-1, 192, 5, 5]               0
           Dropout-9            [-1, 192, 5, 5]               0
           Conv2d-10            [-1, 192, 5, 5]         331,968
             ReLU-11            [-1, 192, 5, 5]               0
           Conv2d-12            [-1, 192, 5, 5]          37,056
             ReLU-13            [-1, 192, 5, 5]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


RuntimeError: ignored